In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append(2*'../')

import os
import glob
from pathlib import Path
from omegaconf import DictConfig
import yaml

import torch
import lightning as L

from rl4co.tasks.rl4co import RL4COLitModule
from rl4co.utils.lightning import load_model_from_checkpoint, clean_hydra_config
from rl4co.tasks.eval import evaluate_policy


/home/botu/botu/Dev/rl4co/env/lib/python3.10/site-packages/torchrl/__init__.py:26: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


## Loading model from checkpoint

In [17]:
checkpoints_path = Path('../../saved_checkpoints/')

exp_name = 'tsp50'
# model_name = 'am-tsp50'
# model_name = "pomo-tsp50"
model_name = "am-tsp50-sm-xl"

checkpoints_path = checkpoints_path / exp_name / model_name

cfg_path = checkpoints_path / 'config.yaml'
# ckpt_path = checkpoints_path / 'epoch_099.ckpt'
ckpt_path = checkpoints_path / 'epoch_499.ckpt'

lit_module = load_model_from_checkpoint(cfg_path, ckpt_path, phase='test')

Unused kwargs: {'params': {'total': 1301888, 'trainable': 1301888, 'non_trainable': 0}}


In [2]:
checkpoints_path = Path('../../saved_checkpoints/')

exp_name = 'tsp50'
# model_name = 'am-tsp50'
# model_name = "pomo-tsp50"
model_name = "symnco-tsp50"

checkpoints_path = checkpoints_path / exp_name / model_name

cfg_path = checkpoints_path / 'config.yaml'
# ckpt_path = checkpoints_path / 'epoch_099.ckpt'
ckpt_path = checkpoints_path / 'epoch_099.ckpt'

lit_module = load_model_from_checkpoint(cfg_path, ckpt_path, phase='test')

Unused kwargs: {'params': {'total': 1334912, 'trainable': 1334912, 'non_trainable': 0}}


In [14]:
# ckpt_path

checkpoints_path = Path('../../saved_checkpoints/')
exp_name = 'tsp50'
model_name = "symnco-tsp50"
checkpoints_path = checkpoints_path / exp_name / model_name

ckpt_path = checkpoints_path / 'epoch_099.ckpt'
loaded = torch.load(ckpt_path, map_location='cpu')
loaded['state_dict'].keys()


odict_keys(['model.policy.encoder.init_embedding.init_embed.weight', 'model.policy.encoder.init_embedding.init_embed.bias', 'model.policy.encoder.layers.0.0.module.Wq', 'model.policy.encoder.layers.0.0.module.Wk', 'model.policy.encoder.layers.0.0.module.Wv', 'model.policy.encoder.layers.0.0.module.Wout', 'model.policy.encoder.layers.0.1.normalizer.weight', 'model.policy.encoder.layers.0.1.normalizer.bias', 'model.policy.encoder.layers.0.2.module.0.weight', 'model.policy.encoder.layers.0.2.module.0.bias', 'model.policy.encoder.layers.0.2.module.2.weight', 'model.policy.encoder.layers.0.2.module.2.bias', 'model.policy.encoder.layers.0.3.normalizer.weight', 'model.policy.encoder.layers.0.3.normalizer.bias', 'model.policy.encoder.layers.1.0.module.Wq', 'model.policy.encoder.layers.1.0.module.Wk', 'model.policy.encoder.layers.1.0.module.Wv', 'model.policy.encoder.layers.1.0.module.Wout', 'model.policy.encoder.layers.1.1.normalizer.weight', 'model.policy.encoder.layers.1.1.normalizer.bias', 

In [15]:
# ckpt_path

checkpoints_path = Path('../../saved_checkpoints/')
exp_name = 'tsp50'
model_name = "am-tsp50"
checkpoints_path = checkpoints_path / exp_name / model_name

ckpt_path = checkpoints_path / 'epoch_099.ckpt'
loaded = torch.load(ckpt_path, map_location='cpu')
loaded['state_dict'].keys()


odict_keys(['model.policy.encoder.init_embedding.init_embed.weight', 'model.policy.encoder.init_embedding.init_embed.bias', 'model.policy.encoder.layers.0.0.module.Wq', 'model.policy.encoder.layers.0.0.module.Wk', 'model.policy.encoder.layers.0.0.module.Wv', 'model.policy.encoder.layers.0.0.module.Wout', 'model.policy.encoder.layers.0.1.normalizer.weight', 'model.policy.encoder.layers.0.1.normalizer.bias', 'model.policy.encoder.layers.0.1.normalizer.running_mean', 'model.policy.encoder.layers.0.1.normalizer.running_var', 'model.policy.encoder.layers.0.1.normalizer.num_batches_tracked', 'model.policy.encoder.layers.0.2.module.0.weight', 'model.policy.encoder.layers.0.2.module.0.bias', 'model.policy.encoder.layers.0.2.module.2.weight', 'model.policy.encoder.layers.0.2.module.2.bias', 'model.policy.encoder.layers.0.3.normalizer.weight', 'model.policy.encoder.layers.0.3.normalizer.bias', 'model.policy.encoder.layers.0.3.normalizer.running_mean', 'model.policy.encoder.layers.0.3.normalizer.

odict_keys(['model.policy.encoder.init_embedding.init_embed.weight', 'model.policy.encoder.init_embedding.init_embed.bias', 'model.policy.encoder.layers.0.0.module.Wq', 'model.policy.encoder.layers.0.0.module.Wk', 'model.policy.encoder.layers.0.0.module.Wv', 'model.policy.encoder.layers.0.0.module.Wout', 'model.policy.encoder.layers.0.1.normalizer.weight', 'model.policy.encoder.layers.0.1.normalizer.bias', 'model.policy.encoder.layers.0.2.module.0.weight', 'model.policy.encoder.layers.0.2.module.0.bias', 'model.policy.encoder.layers.0.2.module.2.weight', 'model.policy.encoder.layers.0.2.module.2.bias', 'model.policy.encoder.layers.0.3.normalizer.weight', 'model.policy.encoder.layers.0.3.normalizer.bias', 'model.policy.encoder.layers.1.0.module.Wq', 'model.policy.encoder.layers.1.0.module.Wk', 'model.policy.encoder.layers.1.0.module.Wv', 'model.policy.encoder.layers.1.0.module.Wout', 'model.policy.encoder.layers.1.1.normalizer.weight', 'model.policy.encoder.layers.1.1.normalizer.bias', 

In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy = lit_module.model.policy.to(device)
policy.eval()
env = lit_module.model.env

In [79]:
test_dataset = lit_module.test_dataset

out_aug = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment", num_augment=20)


Effective batch size: 128 (ratio: 100)
Using automatic batch size: 128


Running greedy_multistart_augment:   0%|          | 0/79 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.6977
Time: 44.7667s


In [63]:
test_dataset = lit_module.test_dataset

# Greedy
out_greedy = evaluate_policy(env, policy, test_dataset, method="greedy")

Effective batch size: 4096 (ratio: 1)
Using automatic batch size: 4096


Running greedy:   0%|          | 0/3 [00:00<?, ?it/s]

Mean reward for greedy: -5.7483
Time: 0.7918s


In [65]:
out_greedy = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment")

Effective batch size: 256 (ratio: 40)
Using automatic batch size: 256


Running greedy_multistart_augment:   0%|          | 0/40 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7003
Time: 17.8459s


In [5]:
test_dataset = lit_module.test_dataset


out_aug = evaluate_policy(env, policy, test_dataset, method="augment", num_augment=20)


Effective batch size: 512 (ratio: 20)
Using automatic batch size: 512


Running augmentation:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for augmentation: -5.7103
Time: 7.9802s


In [5]:
test_dataset = lit_module.test_dataset

# Greedy
out_greedy = evaluate_policy(env, policy, test_dataset, method="greedy")

# Sampling
out_samppling = evaluate_policy(env, policy, test_dataset, method="sampling", samples=50)

# Greedy multistart
out_greedy_ms = evaluate_policy(env, policy, test_dataset, method="greedy_multistart")#, num_starts=20)

# Augment dihedral 8 (same as POMO)
out_augd = evaluate_policy(env, policy, test_dataset, method="augment_dihedral_8")

# Symmetric augment
out_aug = evaluate_policy(env, policy, test_dataset, method="augment", num_augment=50)

# Greedy multistart with augment dihedral 8
out_greedy_ms_augd = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment_dihedral_8")

# Greedy multistart with symmetric augment
out_greddy_ms_aug = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment", num_augment=50)

Effective batch size: 4096 (ratio: 1)
Using automatic batch size: 4096


Running greedy:   0%|          | 0/3 [00:00<?, ?it/s]

Mean reward for greedy: -5.7785
Time: 0.5134s
Effective batch size: 512 (ratio: 50)
Using automatic batch size: 512


Running sampling:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for sampling: -5.7340
Time: 18.9422s
Effective batch size: 4096 (ratio: 5)
Using automatic batch size: 4096


...

Mean reward for augmentation: -5.7048
Time: 19.1155s
Effective batch size: 512 (ratio: 40)
Using automatic batch size: 512


Running greedy_multistart_augment:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7148
Time: 14.8714s
Effective batch size: 128 (ratio: 250)
Using automatic batch size: 128


Running greedy_multistart_augment:   0%|          | 0/79 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7023
Time: 91.0452s
